In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime

In [2]:
# 각 검색어 별 최종 결과인 데이터프레임만 가져오기 위해서 df_list를 선언한다.
df_list = [ele for ele in os.listdir() if bool(re.search("^twitter_[가-힣]+\s*[가-힣]+\.xlsx$", ele))]
print(df_list)

['twitter_택시 타다.xlsx', 'twitter_모빌리티플랫폼.xlsx']


In [3]:
# df_list에 있는 파일들을 하나로 합친다.
df = pd.DataFrame()
for df_name in df_list:
    df = pd.concat([df, pd.read_excel(df_name)])

In [4]:
# 줄바꿈('\n') 문자를 띄어쓰기로 바꾼다.
df['contents'] = df['contents'].str.replace("(\n)", " ")

# content가 NaN(기사 내용이 없음)일 경우는 삭제한다.
mask = df['contents'].notnull()
df = df.loc[mask, :]

# '@아이디' 형태의 태그를 삭제한다.
df['contents'] = df['contents'].str.replace("(@\w+)", "")

# '#문자열' 형태의 해시태그는 삭제한다.
df['contents'] = df['contents'].str.replace("#[가-힣|a-z|A-Z|0-9]+", "")

# 'http:' 형태의 URL 문자열을 삭제한다.
df['contents'] = df['contents'].str.replace("(https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}([-a-zA-Z0-9@:%_\+.~#?&//=]*))", "")

# '이메일 주소'를 삭제한다.
df['contents'] = df['contents'].str.replace("(\w+@\w+((\w+\.\w+\.\w+)|(\w+\.\w+)))", "")

# 띄어쓰기가 2회 이상 되었을 경우에는 띄어쓰기 1회로 변경한다.
df['contents'] = df['contents'].str.replace("(\s{2,})", " ")

# 날짜 형식을 2019년의 경우 제일 앞에 2019년을 붙여준다.
df['dates'] = ["2019년 " + i if not bool(re.search('년', i)) else i for i in df['dates']]

# 날짜 형식을 yyyy.mm.dd.로 변경한다.
df['dates'] = [datetime.datetime.strptime(i, "%Y년 %m월 %d일").strftime("%Y.%m.%d.") for i in df['dates']]

In [5]:
# 최종 결과를 저장한다.
df.to_excel("twitter_crawling_result.xlsx")